## Задание 1

In [3]:
import pandas as pd

In [2]:
!wget https://raw.githubusercontent.com/Yan-Weijia/ML/master/1_intro_regression/weatherHistory.csv

--2020-06-26 10:57:11--  https://raw.githubusercontent.com/Yan-Weijia/ML/master/1_intro_regression/weatherHistory.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16294377 (16M) [text/plain]
Saving to: ‘weatherHistory.csv’

weatherHistory.csv  100%[===================>]  15.54M  37.6MB/s    in 0.4s    

2020-06-26 10:57:12 (37.6 MB/s) - ‘weatherHistory.csv’ saved [16294377/16294377]



In [3]:
weather_hist = pd.read_csv('weatherHistory.csv')

In [4]:
weather_hist.rename({'Formatted Date':'date', 'Temperature (C)':'temp', 'Apparent Temperature (C)':'temp_app',
                    'Precip Type':'weather_type', 'Wind Speed (km/h)':'wind_speed', 'Humidity':'humidity',
                    'Daily Summary':'summary'}, 
                    axis=1, inplace=True)

In [5]:
weather_hist.loc[:, 'date'] = weather_hist.date.apply(pd.to_datetime)

Самый холодный день

In [7]:
# Temperature
coldest_day = weather_hist.loc[weather_hist.temp.idxmin()]
print(str(pd.to_datetime(coldest_day.date).date()), 
      str(coldest_day.temp))

2012-02-10 -21.822222222222226


In [8]:
# Apparent Temperature
coldest_day_app = weather_hist.loc[weather_hist.temp_app.idxmin()]
print(str(pd.to_datetime(coldest_day_app.date).date()), 
      str(coldest_day_app.temp))

2012-02-11 -20.0


самый ветренный день за всю имеющуюся историю:

In [10]:
most_windy_day = weather_hist.loc[weather_hist.wind_speed.idxmax()]
print(str(pd.to_datetime(most_windy_day.date).date()), 
      str(most_windy_day.wind_speed))

2007-01-29 63.8526


сколько раз шел дождь летом 2014 года:

In [9]:
rainy_days = weather_hist[weather_hist.weather_type == 'rain']
rainy_days = rainy_days[[x.date.year == 2014 and x.date.month in (6, 7, 8)
                         for _, x in rainy_days.iterrows()]]
rainy_days = set(rainy_days.date.apply(lambda x: x.date()))

print(len(rainy_days))

92


Сгруппируйтей данные по годам, месяцам и неделям (по отдельности)

In [11]:
weather_hist['year'] = weather_hist.date.apply(lambda x: x.year)
weather_hist['month'] = weather_hist.date.apply(lambda x: x.month)
weather_hist['week'] = weather_hist.date.apply(lambda x: x.week)

самый ветрянный месяц: Март

In [12]:
weather_hist.groupby('month').agg(['mean'])

,temp,temp_app,humidity,wind_speed,Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),year,week
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
month,,,,,,,,,,
1,0.813890,-1.939615,0.850723,11.512816,180.405425,7.830584,0.0,1006.125792,2011.000000,4.255132
2,2.159699,-0.565343,0.813400,12.185543,188.717176,8.731368,0.0,1003.929313,2011.009646,6.948553
3,6.906599,5.074268,0.702966,13.405461,186.538480,10.910450,0.0,1001.551536,2011.000000,11.180105
4,12.756417,12.141164,0.641133,11.893094,186.653157,11.784224,0.0,1009.996332,2011.000000,15.539394
5,16.873692,16.732751,0.691325,10.959337,188.071237,11.892754,0.0,1003.499530,2011.000000,19.894428
6,20.715617,20.742264,0.686470,9.626471,202.780808,11.990266,0.0,1001.883742,2011.000000,24.254545
7,22.963943,22.976086,0.639657,9.639907,202.812248,12.187820,0.0,1008.566431,2010.998900,28.609828
8,22.345031,22.270497,0.635542,8.933431,195.329800,12.455549,0.0,1001.716944,2010.997076,33.032164


самый влажный год: 2010

In [13]:
weather_hist.groupby('year').agg(['mean'])

,temp,temp_app,humidity,wind_speed,Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),month,week
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
year,,,,,,,,,,
2006,11.215365,10.158215,0.767341,10.189852,189.342808,9.758083,0.0,992.543529,6.526826,26.573402
2007,12.135239,11.063135,0.689652,10.825392,194.119292,10.408191,0.0,1001.640226,6.526826,26.433676
2008,12.161876,11.053184,0.701237,11.303897,193.825250,10.267119,0.0,1007.734504,6.514458,26.364071
2009,12.267910,11.069337,0.707247,11.505948,180.496461,10.001370,0.0,1002.608735,6.526826,27.003425
2010,11.202061,10.098643,0.796858,11.015628,192.067509,9.140810,0.0,1004.811891,6.530852,26.593693
2011,11.524453,10.584294,0.736017,9.898262,189.342352,9.507929,0.0,1014.184075,6.526826,26.573402
2012,11.986726,10.753081,0.689500,11.264545,187.755692,10.363108,0.0,999.341481,6.514458,26.503529
2013,11.940719,10.825320,0.754209,10.969389,187.861758,10.892049,0.0,1004.950764,6.526826,26.433562


неделя с самой переменчивой погодой: 5-ая неделя

In [14]:
week_range_data = weather_hist[['temp', 'temp_app', 'wind_speed', 'week']].\
    groupby('week').agg(['max', 'min'])
weather_range = pd.Series([0.] * (len(week_range_data)))
weather_range.index += 1
week_range_data['range'] = weather_range

In [15]:
for feature in ['temp', 'temp_app', 'wind_speed']:
  feature_range = week_range_data[feature]['max'] - week_range_data[feature]['min']
  mean = feature_range.mean()
  std = feature_range.std()
  week_range_data['range'] += (feature_range - mean) / std

In [16]:
week_range_data.head()

temp              temp_app            wind_speed          range
            max        min        max        min        max  min          
week                                                                      
1     12.838889 -13.255556  12.838889 -19.472222    42.7938  0.0 -0.393012
2     15.738889 -15.000000  15.738889 -18.316667    44.6292  0.0  1.949189
3     13.888889 -11.977778  13.888889 -19.961111    46.8349  0.0  0.559251
4     14.772222 -14.088889  14.772222 -22.094444    55.9314  0.0  3.804308
5     14.927778 -17.777778  14.927778 -23.872222    63.8526  0.0  6.882768

In [17]:
week_range_data.range.idxmax()

5

In [18]:
week_range_data.loc[week_range_data.range.idxmax()].range

    6.882768
Name: 5, dtype: float64

## Задание 2

In [31]:
import re
import numpy as np
from scipy.sparse import hstack

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, SGDRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Попробуйте придумать признаки (как минимум 5), которые можно извлечь из текста, помимо тех 4, которые мы уже использовали. Обучите модель на них и проверьте, улучашается ли качество. Опишите признаки, которое придумали и почему считаете, что они должны показывать вероятность покупки.

Побробуйте подобрать параметры векторизатора (TfidfVectorizer или CountVectorizer, или обоих) и параметры алгоритма так, чтобы качество, которое у нас получилось (0.248) улучшилось. Опишите то, что попробовали.

除了我们已经使用的4个符号外，还尝试提出可以从文本中提取的符号（至少5个）。 在它们上训练模型，并检查质量是否正在提高。 描述您想出的迹象，以及为什么您认为它们应该显示购买的可能性。

尝试选择矢量化器的参数（TfidfVectorizer或CountVectorizer或两者）和算法的参数，以便提高获得的质量（0.248）。 描述您尝试过的方法。

In [4]:
data = pd.read_csv('data/data.csv')

### старые признаки

In [5]:
data['text_len'] = data['description'].apply(len)
data['text_len_tokens'] = data['description'].apply(lambda x: len(x.split()))
data['number_of_caps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
data['number_of_nonalphanum'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))

### новые признаки

Количество цифр. 

Цифры могут сделать подробнее описание.

In [7]:
data['digits'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalpha()]))

Количество латинских символов и токенов. 

Обычно они названия брендов или моделей.

In [16]:
data['latin'] = data.description.apply(
    lambda x: len([ch for ch in x if re.search(r'[a-z]', ch.lower()) is not None]))
data['latin_tokens'] = data.description.apply(
    lambda x: len([token for token in x.split() if re.search(r'^[a-z]+$', token.lower()) is not None]))

In [ ]:
Количество кириллических символов и токенов. 

Чем  больше текст, тем подробнее описание.

In [17]:
data['cyrillic'] = data.description.apply(
    lambda x: len([ch for ch in x if re.search(r'[а-яё]', ch.lower()) is not None]))
data['cyrillic_tokens'] = data.description.apply(
    lambda x: len([token for token in x.split() if re.search(r'^[а-яё]+$', token.lower()) is not None]))

Количество символов и токенов в заголовке.

Слишком длинные заголовки не имеют фокуса, а слишком короткие заголовки лишены информации

In [18]:
data['title_len'] = data.title.apply(len)
data['title_len_tokens'] = data.title.apply(lambda x: len(x.split()))

In [19]:
X = data[['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum',
          'digits', 'latin', 'latin_tokens',
          'cyrillic', 'cyrillic_tokens', 'title_len', 'title_len_tokens']].values
y = data.deal_probability.values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [21]:
regressor = ElasticNet(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.26367317664827983

Стало немного лучше.

### CountVectorizer, TfidfVectorizer

нграммы TfidfVectorizer

In [42]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1, 2))
tfidf = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.4, analyzer='char', ngram_range=(3, 4))

X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X = hstack([X_cv, X_tfidf])

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [44]:
regressor = SGDRegressor(max_iter=50, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.25055503172999183

увеличить alpha SGDRegressor

In [45]:
regressor = SGDRegressor(max_iter=50, alpha=1000)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2637259014878597

уменьшить max_df

In [46]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.25, ngram_range=(1, 2))
tfidf = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.25, analyzer='char', ngram_range=(3, 4))

X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X = hstack([X_cv, X_tfidf])

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [48]:
regressor = SGDRegressor(max_iter=50, alpha=1000)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2637259062524335

увеличить min_df

In [49]:
cv = CountVectorizer(max_features=1000, min_df=10, max_df=0.25, ngram_range=(1, 2))
tfidf = TfidfVectorizer(max_features=1000, min_df=10, max_df=0.25, analyzer='char', ngram_range=(3, 4))

X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X = hstack([X_cv, X_tfidf])

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [51]:
regressor = SGDRegressor(max_iter=50, alpha=1000)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.26372637627019296

Результаты стали лучше.